<a href="https://colab.research.google.com/github/torstenwerner/Agentic_Design_Patterns/blob/main/notebooks/Chapter05_Tool_Use_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-community langchain-google-genai langgraph

In [2]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_PAID')

In [24]:
import os
import asyncio
from typing import List

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_agent
from IPython.display import display, Markdown

# --- Configuration ---
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0)

# --- Define a Tool ---
@tool
def search_information(query: str) -> str:
    """
    Provides factual information on a given topic. Use this tool to find answers to questions
    like 'What is the capital of France?' or 'What is the weather in London?'.
    """
    print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
    # Simulate a search tool with a dictionary of predefined results.
    simulated_results = {
        "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
        "capital of france": "The capital of France is Paris.",
        "population of earth": "The estimated population of Earth is around 8 billion people.",
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
        "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
    }
    result = simulated_results.get(query.lower().replace("current weather", "weather"), simulated_results["default"])
    print(f"--- TOOL RESULT: {result} ---")
    return result

tools = [search_information]


# --- Create a Tool-Calling Agent ---
agent_prompt = "You are a helpful assistant."
agent_executor = create_agent(llm, tools, system_prompt=agent_prompt)

async def run_agent_with_tool(query: str):
    """Invokes the agent executor with a query and prints the final response."""
    print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
    try:
        inputs = {"messages": [("user", query)]}
        response = await agent_executor.ainvoke(inputs)
        print("\n--- ✅ Final Agent Response ---")
        final_message = response["messages"][-1]
        display(Markdown(final_message.content[0]["text"]))
    except Exception as e:
        print(f"\n🛑 An error occurred during agent execution: {e}")

async def main():
    """Runs all agent queries concurrently."""
    tasks = [
        run_agent_with_tool("What is the capital of France?"),
        run_agent_with_tool("What's the weather like in London?"),
        run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
    ]
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    # Run all async tasks in a single event loop.
    await main()


--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---

--- 🛠️ Tool Called: search_information with query: 'capital of France' ---
--- TOOL RESULT: The capital of France is Paris. ---

--- 🛠️ Tool Called: search_information with query: 'current weather in London' ---
--- TOOL RESULT: The weather in London is currently cloudy with a temperature of 15°C. ---

--- ✅ Final Agent Response ---


The weather in London is currently cloudy with a temperature of 15°C.


--- ✅ Final Agent Response ---


The capital of France is **Paris**.


--- ✅ Final Agent Response ---


Dogs are fascinating creatures with a history and set of biological traits that make them unique in the animal kingdom. Here are some of the most interesting things to know about them:

### 1. Their Sense of Smell is "Superpowered"
A dog’s sense of smell is 10,000 to 100,000 times more acute than a human's. To put that in perspective, while a human might notice if a teaspoon of sugar has been added to their coffee, a dog could detect that same teaspoon of sugar in a million gallons of water (about two Olympic-sized swimming pools). 

### 2. They "See" with Their Noses
Dogs have a special organ called the **vomeronasal organ** (or Jacobson's organ) located in the roof of their mouth. This allows them to "taste" smells, helping them detect pheromones and chemicals that tell them about the health, mood, and gender of other animals.

### 3. Unique "Fingerprints"
Just like humans have unique fingerprints, every dog has a **unique nose print**. No two dogs have the same pattern of ridges and creases on their noses. Some kennel clubs and insurance companies even use nose prints to identify lost dogs.

### 4. Tail Wagging is a Language
A wagging tail doesn't always mean a dog is happy. It is a complex communication tool:
*   **Wagging to the right:** Usually indicates the dog is relaxed or happy.
*   **Wagging to the left:** Often indicates anxiety or fear.
*   **Low, slow wag:** Indicates insecurity.
*   **High, stiff wag:** Indicates the dog is alert or potentially aggressive.

### 5. They Are as Smart as Toddlers
Studies in canine intelligence have found that the average dog is about as smart as a **two-year-old human child**. They can understand roughly 165 words and gestures, and "super dogs" (those in the top 20% of intelligence) can learn up to 250 words.

### 6. Ancient Companions
Dogs were the very first species to be domesticated by humans. While the exact timeline is debated, evidence suggests it happened between **15,000 and 30,000 years ago**. They evolved from a now-extinct species of wolf that chose to live near human camps for food scraps.

### 7. Why They Curl Up to Sleep
When dogs sleep in a ball, it’s an ancestral instinct. Curling up protects their vital organs from predators and helps them conserve body heat. 

### 8. They Dream Like Us
If you’ve ever seen your dog twitching or "running" in their sleep, they are likely dreaming. Research shows that dogs have similar brain wave patterns to humans during sleep and go through REM (Rapid Eye Movement) cycles, which is when dreaming occurs.

### 9. Sweat Glands
Dogs don’t sweat through their skin like humans do. They only have sweat glands in their **paw pads**. This is why panting is so important—it’s their primary way of cooling down through evaporation on their tongue.

### 10. Diversity of Breeds
There are over **340 recognized dog breeds** in the world. Despite the massive physical difference between a tiny Chihuahua and a massive Great Dane, they are all the same species (*Canis lupus familiaris*) and are biologically almost identical.

In [ ]:
!pip install -q -U "dotenv==0.9.9" "langchain-google-genai==2.1.8" "crewai==0.150.0" "google-adk==1.8.0"

In [ ]:
import os, getpass
import asyncio
import nest_asyncio
from typing import List
from dotenv import load_dotenv
import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

# UNCOMMENT
# Prompt the user securely and set API keys as an environment variables
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

try:
   # A model with function/tool calling capabilities is required.
   llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
   print(f"✅ Language model initialized: {llm.model}")
except Exception as e:
   print(f"🛑 Error initializing language model: {e}")
   llm = None

# --- Define a Tool ---
@langchain_tool
def search_information(query: str) -> str:
   """
   Provides factual information on a given topic. Use this tool to find answers to phrases
   like 'capital of France' or 'weather in London?'.
   """
   print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
   # Simulate a search tool with a dictionary of predefined results.
   simulated_results = {
       "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
       "capital of france": "The capital of France is Paris.",
       "population of earth": "The estimated population of Earth is around 8 billion people.",
       "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
       "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting."
   }
   result = simulated_results.get(query.lower(), simulated_results["default"])
   print(f"--- TOOL RESULT: {result} ---")
   return result

tools = [search_information]

# --- Create a Tool-Calling Agent ---
if llm:
   # This prompt template requires an `agent_scratchpad` placeholder for the agent's internal steps.
   agent_prompt = ChatPromptTemplate.from_messages([
       ("system", "You are a helpful assistant."),
       ("human", "{input}"),
       ("placeholder", "{agent_scratchpad}"),
   ])

   # Create the agent, binding the LLM, tools, and prompt together.
   agent = create_tool_calling_agent(llm, tools, agent_prompt)

   # AgentExecutor is the runtime that invokes the agent and executes the chosen tools.
   # The 'tools' argument is not needed here as they are already bound to the agent.
   agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)

async def run_agent_with_tool(query: str):
   """Invokes the agent executor with a query and prints the final response."""
   print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
   try:
       response = await agent_executor.ainvoke({"input": query})
       print("\n--- ✅ Final Agent Response ---")
       print(response["output"])
   except Exception as e:
       print(f"\n🛑 An error occurred during agent execution: {e}")

async def main():
   """Runs all agent queries concurrently."""
   tasks = [
       run_agent_with_tool("What is the capital of France?"),
       run_agent_with_tool("What's the weather like in London?"),
       run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response
   ]
   await asyncio.gather(*tasks)

nest_asyncio.apply()
asyncio.run(main())